In [1]:
import cv2
import json
import math
import os
import os.path
import re
import tempfile
import airsim
import numpy as np
from PIL import Image
from matplotlib import pyplot
import quickstart as util
import utils_KITTI as utilkitti

Using TensorFlow backend.
/home/islab/.local/share/virtualenvs/depth_prediction-uX_uZt9E/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/islab/.local/share/virtualenvs/depth_prediction-uX_uZt9E/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/islab/.local/share/virtualenvs/depth_prediction-uX_uZt9E/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) /

In [2]:
# 資料路徑
DATA_PATH = '/home/islab/Documents/drone/depth_prediction/data/random/data/2020-03-30-01-10/depth_planner/2020-03-30-01-19-14'
GREY_PIC_PATH = '/home/islab/Documents/drone/depth_prediction/result/vis_depth/2020-03-30-01-19-14/visualized_depth'
RESULT_PATH = '/home/islab/Documents/drone/depth_prediction/depth_differ/result/2020-03-30-01-19-14'

# Model Input 解析度設定
INPUT_HEIGHT = 320
INPUT_WIDTH = 384

#
ARROW_SIZE = 50

In [ ]:
result_image_folder_path = os.path.join(RESULT_PATH, '3D_analysis')
if not os.path.exists(result_image_folder_path):
    os.makedirs(result_image_folder_path)
for i in range(50):
    # 讀取影像
    n = "%03d" % (i+1)
    depth, scale = utilkitti.read_pfm(os.path.join(DATA_PATH, str(n)+'.pfm'))
    X = np.zeros((INPUT_HEIGHT, INPUT_WIDTH))
    Y = np.zeros((INPUT_HEIGHT, INPUT_WIDTH))
    im = Image.open(os.path.join(GREY_PIC_PATH, str(n)+'.png'))
    im_rgb = im.convert("RGB")
    danger = False
    for j in range(INPUT_HEIGHT):
        for k in range(INPUT_WIDTH):
            Y[j, k] = depth[j, k] * (j - 160) / 192
            X[j, k] = depth[j, k] * (k - 192) / 192
            if abs(X[j, k]) <= 0.5 and abs(Y[j, k]) <= 0.25:
                if depth[j, k] < 10:
                    danger = True
                    
                    #im_rgb.putpixel((k, j), (150, 0, 0))
                #else:
                    #im_rgb.putpixel((k, j), (256, 0, 0))
                    #danger = True
    #
    #WN N EN E ES S WS W
    safe_count = np.array([0, 0, 0, 0, 0, 0, 0, 0])
    safe_weight_count = np.array([0, 0, 0, 0, 0, 0, 0, 0])
    danger_count = np.array([0, 0, 0, 0, 0, 0, 0, 0])
    danger_prop = np.array([0, 0, 0, 0, 0, 0, 0, 0])
    safe_dir = 0
    max_safe_weight = 0
    danger_area_count = 0
    
    if danger:
        for j in range(INPUT_HEIGHT):
            for k in range(INPUT_WIDTH):
                if abs(X[j, k]) <= 2.5 and abs(Y[j, k]) <= 1.25:
                    ind_1 = 0
                    ind_2 = 0
                    
                    if X[j, k] <= 0.0 and Y[j, k] >= 0.0:
                        ind_1 = 0
                        if abs(X[j, k]/Y[j, k]) > 2:
                            ind_2 = 7
                        else:
                            ind_2 = 1
                    if X[j, k] >= 0.0 and Y[j, k] >= 0.0:
                        ind_1 = 2
                        if abs(X[j, k]/Y[j, k]) > 2:
                            ind_2 = 3
                        else:
                            ind_2 = 1
                    if X[j, k] >= 0.0 and Y[j, k] <= 0.0:
                        ind_1 = 4
                        if abs(X[j, k]/Y[j, k]) > 2:
                            ind_2 = 3
                        else:
                            ind_2 = 5
                    if X[j, k] <= 0.0 and Y[j, k] <= 0.0:
                        ind_1 = 6
                        if abs(X[j, k]/Y[j, k]) > 2:
                            ind_2 = 7
                        else:
                            ind_2 = 5
                        
                    if depth[j, k] > 10:
                        im_rgb.putpixel((k, j), (0, 256, 0))
                        safe_weight_count[ind_1] += (depth[j, k] * depth[j, k])
                        safe_count[ind_1] += 1
                        safe_weight_count[ind_2] += (depth[j, k] * depth[j, k])
                        safe_count[ind_2] += 1
                    else:
                        im_rgb.putpixel((k, j), (256, 0, 0))
                        danger_count[ind_1] += 1
                        danger_count[ind_2] += 1
        for j in range(8):
            danger_prop[j] = danger_count[j] / (danger_count[j] + safe_count[j])
            if danger_prop[j] < 0.5 and safe_weight_count[j] > max_safe_weight:
                max_safe_weight = safe_weight_count[j]
                safe_dir = j
            else:
                danger_area_count += 1  
    
    pyplot.clf()
    pyplot.axis('off')
    pyplot.imshow(im_rgb)
    if danger:
        #max_area = np.argmax(counter)
        if safe_dir == 0:
            pyplot.arrow(INPUT_WIDTH/2, INPUT_HEIGHT/2, -ARROW_SIZE, -ARROW_SIZE, color='b', head_width=10)
        elif safe_dir == 1:
            pyplot.arrow(INPUT_WIDTH/2, INPUT_HEIGHT/2, 0, -ARROW_SIZE, color='b', head_width=10)
        elif safe_dir == 2:
            pyplot.arrow(INPUT_WIDTH/2, INPUT_HEIGHT/2, ARROW_SIZE, -ARROW_SIZE, color='b', head_width=10)
        elif safe_dir == 3:
            pyplot.arrow(INPUT_WIDTH/2, INPUT_HEIGHT/2, -ARROW_SIZE, 0, color='b', head_width=10)
        elif safe_dir == 7:
            pyplot.arrow(INPUT_WIDTH/2, INPUT_HEIGHT/2, ARROW_SIZE, 0, color='b', head_width=10)
        elif safe_dir == 6:
            pyplot.arrow(INPUT_WIDTH/2, INPUT_HEIGHT/2, -ARROW_SIZE, ARROW_SIZE, color='b', head_width=10)
        elif safe_dir == 5:
            pyplot.arrow(INPUT_WIDTH/2, INPUT_HEIGHT/2, 0, ARROW_SIZE, color='b', head_width=10)
        elif safe_dir == 4:
            pyplot.arrow(INPUT_WIDTH/2, INPUT_HEIGHT/2, ARROW_SIZE, ARROW_SIZE, color='b', head_width=10)
    # 儲存結果
    pyplot.savefig(os.path.join(result_image_folder_path, str(n)+'.png'))
    print(os.path.join(result_image_folder_path, str(n)+'.png'))
    print(safe_weight_count)